# import Statements

In [184]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint


In [185]:
df = pd.read_csv("train.csv")
df = df.rename(columns={"left": "label"})

In [186]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,label,promotion_last_5years,sales,salary
0,0.10,0.90,7,286,4,0,1,0,sales,low
1,0.89,0.93,4,249,3,0,0,0,sales,low
2,0.38,0.50,2,132,3,0,1,0,accounting,low
3,0.95,0.71,4,151,4,0,0,0,sales,medium
4,0.84,0.84,5,163,3,0,0,0,technical,low


In [187]:
len(df)
df.dtypes

satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
label                      int64
promotion_last_5years      int64
sales                     object
salary                    object
dtype: object

In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 10 columns):
satisfaction_level       11238 non-null float64
last_evaluation          11238 non-null float64
number_project           11238 non-null int64
average_montly_hours     11238 non-null int64
time_spend_company       11238 non-null int64
Work_accident            11238 non-null int64
label                    11238 non-null int64
promotion_last_5years    11238 non-null int64
sales                    11238 non-null object
salary                   11238 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 878.0+ KB


# Training and Validation data split

In [189]:
def train_validate_split(df,test_size):
    if isinstance(test_size,float):
        test_size = round(test_size*len(df))
        
    indices=df.index.tolist()
    validate_indices = random.sample(population=indices,k=test_size)
    
    validate_df=df.loc[validate_indices]
    train_df=df.drop(validate_indices)
    return train_df,validate_df

In [190]:
# random.seed(0)
train_df,validate_df=train_validate_split(df,test_size=0.2)

# Helper Functions

In [191]:
data = df.values
validate_df.head()
name_list=list(df)
column_list=[1,2,3,4,5,7,8,9]
# diction=dict()
# diction["salary"]=9
# diction["sales"]=8
# diction["promotion_last_5years"]=7
# diction["Work_accident"]=5

# Check_Purity (Unique Values Check)

In [192]:
def check_purity(data):
    unique_vals=np.unique(data[:,6])
    if len(unique_vals)==1:
        return unique_vals[0]
    return -1

# Classify (It will return data which is offen)

In [193]:
def classify_data(data):
    
    label_column = data[:, 6]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

# Potential Split

In [194]:
def get_potential_splits(data):
    
    potential_splits = {}
    n_column=column_list
    for column_index in range(n_columns):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        unique_values.sort()
        
        type_of_feature = FEATURE_TYPES[column_index]
        if type_of_feature == "continuous":
            potential_splits[column_index] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[column_index].append(potential_split)
        
        # feature is categorical
        # (there need to be at least 2 unique values, otherwise in the
        # split_data function data_below would contain all data points
        # and data_above would be empty)
        elif len(unique_values) > 1:
            potential_splits[column_index] = unique_values
    
    return potential_splits

# Split Data

In [195]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above


# Lowest Overall Entropy?

In [196]:
def calculate_entropy(data):
    
    label_column = data[:, 6]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [197]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [198]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

# Entropy Calculation

In [199]:
def entropy_cal(training_data,column_no):
    unique_vals,unique_val_count=np.unique(training_data[:,column_no],return_counts=True)
    total_len=len(training_data)
    entropy_sum=0
    dic=dict()
    for j in range(len(unique_vals)):
        true=0
        false=0
        for i in training_data:
            if (i[6]==1)and(unique_vals[j]==i[column_no]):
                true+=1
            elif (i[6]==0)and(unique_vals[j]==i[column_no]):
                false+=1

        p_true=true/(true+false)
        p_false=false/(true+false)
        total=unique_val_count[j]
#         print(total)
        if p_true and p_false:
            entropy=(-1)*(p_true*(math.log2(p_true))+p_false*(math.log2(p_false)))
            
            dic[unique_vals[j]]=entropy
            
#         print(entropy)
            entropy_weighted=(entropy*total)/total_len
            entropy_sum+=entropy_weighted
#     print(entropy_sum)
    return (entropy_sum,dic)

In [200]:
def overall_entropy(training_data):
    true=0
    false=0
    for i in training_data[:,6]:
        if i==1:
            true=true+1
        elif i==0:
            false=false+1
    p_true=true/(true+false)
    p_false=false/(true+false)
    entropy_of_training_data=(-1)*(p_true*(math.log2(p_true))+p_false*(math.log2(p_false)))
    print(entropy_of_training_data)
    
    return entropy_of_training_data

In [201]:
def information_gain(training_data):
    entropy_of_training_data=overall_entropy(training_data);
    
    entropy_of_unique_vals=dict()
    entropies_list=dict()
    for i in col_list:
        temp,entropy_of_unique_vals[i]=entropy_cal(training_data,i)
        entropies_list[i]=temp

#     min
#     for i in col_list:
#     sorted_x = sorted(entropies_list.items(), key=operator.itemgetter(0))
    sorted_x = sorted(entropies_list.items(), key=lambda x: x[1])
    print()
    selected=sorted_x[0][0]    
    gain=sorted_x[0][1]

#     info_gain=entropy_of_training_data-gain
    print()
    print(entropy_of_unique_vals)
#     print(info_gain)
    return selected

# Determine_type_of_feature

In [202]:
def determine_type_of_feature(df):
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

# Algo

In [203]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree


In [204]:
tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree)

0
